# Auto Correct

# Part 1: Data Preprocessing 

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import re
import string

In [2]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub(" و ", " ", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [3]:
def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You just have to read it in. 
    Output: 
        words: a list containing all the words in the corpus (text file you read) in lower case. 
    """
    
    df = pd.read_csv(file_name, sep='\t', usecols=['review']) 
    text = ' '.join(df["review"])  
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    text = remove_punctuations(text)
    words = text.split() 

    return words

In [4]:
word_l = process_data('./data/arabic_reviews.tsv')
vocab = set(word_l)

# Part 2: Get Probability

In [5]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    
    word_count_dict = {}
    word_count_dict = Counter(word_l)
    return word_count_dict

In [6]:
word_count_dict = get_count(word_l)

In [7]:
word_count_dict['الله']

98723

In [8]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}  
    
    m = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / m
    return probs

In [20]:
probs = get_probs(word_count_dict)

# Part 3: Word Manipulation

In [9]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    
    delete_l = []
    split_l = []
    
    for c in range(len(word)):
        split_l.append((word[:c],word[c:]))
    for a,b in split_l:
        delete_l.append(a+b[1:])          

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [10]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    '''
    def swap(c, i, j):
        c = list(c)
        c[i], c[j] = c[j], c[i]
        return ''.join(c)
    
    switch_l = []
    split_l = []
    len_word=len(word)
    
    for c in range(len_word):
        split_l.append((word[:c],word[c:]))
    switch_l = [a + b[1] + b[0] + b[2:] for a,b in split_l if len(b) >= 2]      
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [12]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = list('ابتثجحخدذرزسشصضطظعغفقكلمنهويء')
    replace_l = []
    split_l = []
    
    
    for c in range(len(word)):
        split_l.append((word[0:c],word[c:]))
    replace_l = [a + l + (b[1:] if len(b)> 1 else '') for a,b in split_l if b for l in letters]
    replace_set=set(replace_l)    
    replace_set.remove(word)
    
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [14]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = list('ابتثجحخدذرزسشصضطظعغفقكلمنهويء')
    insert_l = []
    split_l = []
    
    for c in range(len(word)+1):
        split_l.append((word[0:c],word[c:]))
    insert_l = [ a + l + b for a,b in split_l for l in letters]

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [15]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """
    
    edit_one_set = set()
        
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return edit_one_set

In [16]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = set()
    edit_one = edit_one_letter(word,allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w,allow_switches=allow_switches)
            edit_two_set.update(edit_two)

    return edit_two_set

In [17]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    
    suggestions = list(edit_one_letter(word).intersection(vocab) 
                       or edit_two_letters(word).intersection(vocab))
    
    n_best = sorted([[s,probs.get(s,0)] for s in suggestions], key=lambda x:x[1], reverse=True)
    
    if verbose: print("suggestions = ", suggestions)

    return n_best[:n]

# Part 4: Minimum edit distance

In [18]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''

    m = len(source)
    n = len(target)


    D = np.zeros((m+1, n+1), dtype=int) 

    
    for row in range(1,m+1):
        D[row,0] = D[row-1,0] + del_cost
        
        
    for col in range(1,n+1):
        D[0,col] = D[0,col-1] + ins_cost
        

    for row in range(1,m+1): 
        

        for col in range(1,n+1):
            
            r_cost = rep_cost
            
            if source[row-1] == target[col-1]:
                r_cost = 0
                
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])
          
    med = D[m,n]
    
    return D, med

# Test It :D

In [21]:
source = "كمبيوت"

targets = get_corrections(source, probs, vocab, 5, verbose=False)

for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1) 
    print(f'source : {source} \ntarget : {t[0]}\nprobability : {t[1]}\n')

source : كمبيوت 
target : كمبيوتر
probability : 1.8410279411998418e-06

source : كمبيوت 
target : كبيوت
probability : 1.3536970155881188e-07



# Part 5: Dump computed Probabilities

In [48]:
import json

with open('./data/probs.json', 'w', encoding='utf-8') as outfile:
    json.dump(probs, outfile,  ensure_ascii=False)

### Read file

In [49]:
with open('./data/probs.json', encoding='utf-8') as infile:
    data_loaded = json.load(infile) 
    vocab_loaded = data_loaded.keys()

In [50]:
source = "القهو"

targets = get_corrections(source, probs, vocab, 3, verbose=False)

for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1) 
    print(f'source : {source} \ntarget : {t[0]}\nprobability : {t[1]}\n')

source : القهو 
target : القوه
probability : 0.00011108437709916104

source : القهو 
target : القهر
probability : 4.096287169169648e-05

source : القهو 
target : القهوه
probability : 3.292191141910305e-05

